In [226]:
#pip install seaborn

In [ ]:
import random
from enum import Enum

import mesa
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

print(mesa.__version__)

In [228]:
# Enum to track the status of customer orders
class OrderStatus(Enum):
    WAITING = 0    # Customer hasn't ordered yet
    ORDERED = 1    # Order has been placed
    PREPARING = 2  # Food is being prepared
    SERVED = 3     # Food has been delivered

# Available food options in the restaurant
food_options = ["vegetarian", "meat", "gluten free"]

In [229]:
class CustomerAgent(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        # Initialize customer properties
        self.active = True
        self.food_preference = random.choice(food_options)
        self.waiting_time = 0                         # Time spent waiting
        self.order_status = OrderStatus.WAITING       # Current order status
        self.satisfaction = 100                         # Overall satisfaction (0-100)
        self.tip = 0                                 # Amount of tip given
        self.rating = 0                              # Rating given (1-5)
        self.assigned_waiter = None                  # Reference to assigned waiter

    def step(self):
        # Increment waiting time if not served yet
        if self.order_status != OrderStatus.SERVED:
            self.waiting_time += 1

    def order_dish(self, waiter):
        # Place order with waiter if not already ordered
        if self.order_status == OrderStatus.WAITING:
            self.order_status = OrderStatus.ORDERED
            self.assigned_waiter = waiter
            return self.food_preference

    def rate_experience(self):
        # Calculate satisfaction based on waiting time
        self.satisfaction = max(0, 100 - self.waiting_time)
        # Convert satisfaction to 1-5 star rating
        self.rating = max(1, self.satisfaction / 20)
        # Calculate tip based on satisfaction
        self.tip = self.satisfaction / 10
        self.active = False



In [230]:
class WaiterAgent(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        # Initialize waiter properties
        self.model = model
        self.busy = False                # Current serving status
        self.current_orders = {}         # List of (customer, order) tuples
        self.tips = 0                    # Total tips received
        self.avg_rating = 0              # Average rating from customers
        self.ratings_count = 0           # Number of ratings received
        self.served_customers = 0        # Total customers served

    def calculate_closest_customer(self, ordered):
        for customer in self.model.customers:
            if customer in self.current_orders.keys() and ordered:
                return customer
            elif customer not in self.current_orders.keys() and not ordered:
                return customer

    def step(self):
        if self.busy:
            customer = self.calculate_closest_customer(ordered=True)
            print(customer)
            self.serve_dish(customer)
        else:
            self.take_order(self.calculate_closest_customer(ordered=False))

    def take_order(self, customer):
        # Take order from customer if waiter is available
        if not self.busy:
            order = customer.order_dish(self)
            if order:
                self.current_orders.update({customer: order})
                self.busy = True
                return True
        return False

    def serve_dish(self, customer):
        # Serve food to customer and collect feedback
        if customer in self.current_orders.keys():
            customer.order_status = OrderStatus.SERVED
            # Remove served customer from current orders
            del self.current_orders[customer]
            self.busy = len(self.current_orders) > 0
            self.served_customers += 1

            # Update waiter's performance metrics
            customer.rate_experience()
            rating = customer.rating
            tip = customer.tip
            self.tips += tip
            self.ratings_count += 1
            self.avg_rating = ((self.avg_rating * (self.ratings_count - 1)) + rating) / self.ratings_count
            

In [231]:
class ManagerAgent(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        # Initialize manager properties
        self.food_inventory = {option: 100 for option in food_options}  # Initial stock
        # Track daily statistics
        self.daily_stats = {
            'total_customers': 0,        # Total customers in restaurant
            'avg_waiting_time': 0,       # Average customer waiting time
            'active_waiters': 0,         # Number of available waiters
            'profit': 0                  # Daily profit
        }    

    def step(self):
        # Update daily statistics
        model = self.model
        self.daily_stats['total_customers'] = len(model.customers)
        self.daily_stats['active_waiters'] = len([w for w in model.waiters if not w.busy])
        self.daily_stats['avg_waiting_time'] = np.mean([c.waiting_time for c in model.customers])

    def order_food(self, food_type, amount):
        # Replenish food inventory
        self.food_inventory[food_type] += amount

    def calculate_profit(self):
        # Calculate daily profit considering various costs
        total_sales = sum(w.tips for w in self.model.waiters)
        staff_costs = len(self.model.waiters) * 10  # Fixed cost per waiter
        food_costs = sum(100 - amount for amount in self.food_inventory.values())
        self.daily_stats['profit'] = total_sales - (staff_costs + food_costs)

In [232]:
class RestaurantModel(mesa.Model):
    def __init__(self, n_customers, n_waiters, seed=None):
        super().__init__(seed=seed)

        # Create agents
        CustomerAgent.create_agents(model=self, n=n_customers)
        WaiterAgent.create_agents(model=self, n=n_waiters)
        ManagerAgent.create_agents(model=self, n=1)

        self.customers = []
        self.waiters = []
        self.managers = []

        for agent in self.agents:
            if isinstance(agent, CustomerAgent):
                self.customers.append(agent)
            elif isinstance(agent, WaiterAgent):
                self.waiters.append(agent)
            elif isinstance(agent, ManagerAgent):
                self.managers.append(agent)

        print(self.customers)
        print(self.waiters)
        print(self.managers)

        # Set up data collection for model metrics
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "Customer_Count": lambda m: self.get_active_costomers_count(m.customers),
                "Average_Wait_Time": lambda m: np.mean([c.waiting_time for c in m.customers]),
                "Average_Customer_Satisfaction": lambda m: np.mean([c.satisfaction for c in m.customers]),
                "Profit": lambda m: m.managers[0].daily_stats['profit'],
                "Customer_Info": lambda m: self.get_customer_info(m.customers)
            }
        )
    
    def get_customer_info(self, customers):
        c_nr = 0
        c_infos = []
        for customer in customers:
            c_info = {}
            c_info['customer_nr'] = c_nr
            c_info['active'] = 1 if customer.active == True else 0
            c_info['waiting_time'] = customer.waiting_time
            c_info['order_status'] = customer.order_status.value
            c_info['satisfaction'] = customer.satisfaction
            c_infos.append(c_info)
            c_nr += 1
        return c_infos

    def get_active_costomers_count(self, customers):
        return len([customer for customer in customers if customer.active == True])

    def step(self):
        # Collect data and execute agent steps in random order
        self.datacollector.collect(self)
        self.agents.shuffle_do("step")

In [ ]:
params = {"n_waiters":2, "n_customers":5}

results = mesa.batch_run(
    RestaurantModel,
    parameters=params,
    iterations=5,
    max_steps=10,
    number_processes=1,
    data_collection_period=1,
    display_progress=True,
)
print(results)

In [ ]:
import plotly.express as px

df = pd.DataFrame(results)
data_grouped = df.groupby(['Step']).agg(
    mean_customer_count = ('Customer_Count', 'mean'),
    mean_waiting_time = ('Average_Wait_Time', 'mean'),
    mean_customer_satisfaction = ('Average_Customer_Satisfaction', 'mean'),
    mean_profit = ('Profit', 'mean')).reset_index()
data_grouped
#fig = px.line(df, x="year", y="lifeExp", color='country')
#fig.show()

In [ ]:
# Plotting the data in one plot
fig, ax1 = plt.subplots(figsize=(15, 10))

ax1.plot(data_grouped['Step'], data_grouped['mean_customer_count'], marker='o', label='Customer Count')
ax1.plot(data_grouped['Step'], data_grouped['mean_waiting_time'], marker='o', label='Average Wait Time')
ax1.plot(data_grouped['Step'], data_grouped['mean_customer_satisfaction'], marker='o', label='Average Customer Satisfaction')
ax1.plot(data_grouped['Step'], data_grouped['mean_profit'], marker='o', label='Profit')

ax1.set_title('Model Output Data over Steps')
ax1.set_xlabel('Step')
ax1.set_ylabel('Values')
ax1.legend()

plt.show()

In [245]:
import plotly.express as px
df = px.data.tips()
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [246]:
data_first_run = df[df["RunId"]==0]
customer_infos_dict = dict(data_first_run["Customer_Info"])
customer_infos_list = [{**item, 'step': k} for k, v in customer_infos_dict.items() for item in v]
customer_infos_df = pd.DataFrame(customer_infos_list)
customer_infos_df.sort_values(by=["customer_nr", "step"])

KeyError: 'RunId'

In [257]:
plots = ['active','waiting_time','order_status','satisfaction']

import plotly.express as px
for plot in plots:
    fig = px.histogram(customer_infos_df, x="step", y=plot,
                color='customer_nr', barmode='group',
                height=400, nbins=len(customer_infos_df['step'].unique()))
    fig.show()